# Урок 5. Part-of-Speech разметка, NER, извлечение отношений

**Задание 1. Написать теггер на данных с русским языком.**

1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации.
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов.
3. сравнить все реализованные методы, сделать выводы.


**Задание 2. Проверить, насколько хорошо работает NER.**

Данные брать из Index of /pub/named_entities
1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы.
   1. передаём в сетку токен и его соседей.
   2. передаём в сетку только токен.
   3. свой вариант.
3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

### Задание 1.

In [2]:
# !pip install pyconll corus razdel spacy --break-system-packages
# !pip install -U spacy
# !python -m spacy info
# !python -m spacy download ru_core_news_sm 

In [3]:
# !wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
# !wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

In [5]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.tokenize import word_tokenize
import matplotlib
import pyconll
import corus
from corus import load_ne5
import re
import spacy
from spacy import displacy

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score
from sklearn import model_selection, preprocessing, linear_model

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from razdel import tokenize
from corus import load_ne5

import spacy
from spacy import displacy
import ru_core_news_sm
from spacy.lang.ru.examples import sentences 
from spacy.lang.ru import Russian

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, RegexpTagger

import warnings
warnings.filterwarnings("ignore")

data_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
data_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

fdata_train = []
for sent in data_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in data_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in data_test[:]:
    fdata_sent_test.append([token.form for token in sent])

2023-09-29 13:56:38.661026: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-09-29 13:56:38.661121: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-09-29 13:56:38.661150: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-29 13:56:38.669305: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 13:56:39.487836: W tensorflow/compiler/

### Проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации

In [6]:
default_tagger = nltk.DefaultTagger('NN')
default_acc = default_tagger.evaluate(fdata_test)

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'nltk: {round(default_acc, 3)}')
print(f'Unigram: {round(unigram_acc, 3)}')
print(f'Bigram: {round(bigram_acc, 5)}')
print(f'Trigram: {round(trigram_acc, 3)}')
print(f'Bigram + Unigram: {round(bigram_unigram_acc, 5)}')
print(f'Trigram + Bigram + Unigram: {round(trigram_bigram_unigram_acc, 5)}')

nltk: 0.0
Unigram: 0.824
Bigram: 0.60939
Trigram: 0.178
Bigram + Unigram: 0.82928
Trigram + Bigram + Unigram: 0.82914


### Объединённый теггер

In [7]:
def union_taggers(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

backoff = DefaultTagger('NN') 
tag = union_taggers(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.827905462595221

### Написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов.

In [8]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(' ' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [9]:
train_tok[:7], train_label[:7]

(['Анкета', '.', 'Начальник', 'областного', 'управления', 'связи', 'Семен'],
 ['NOUN', 'PUNCT', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PROPN'])

In [10]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [11]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [12]:
vectorizers = [CountVectorizer(ngram_range=(1, 5), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 5), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels, pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels, pred)
    accuracy_scores.append(acc)
    # print(vectorizer)
    # print(classification_report(test_enc_labels, pred, target_names=le.classes_))

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

result_model = pd.DataFrame({
    'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
    'f1_score': f1_scores,
    'Accuracy': accuracy_scores
})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(1, 5))",0.938119,0.939469
1,"TfidfVectorizer(analyzer='char', ngram_range=(1, 5))",0.927087,0.929318
8,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 5))",0.909308,0.912540
7,"HashingVectorizer(analyzer='char', n_features=3000, ngram_range=(1, 5))",0.898584,0.902110
6,"HashingVectorizer(analyzer='char', n_features=2000, ngram_range=(1, 5))",0.892840,0.896243
2,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 5))",0.866822,0.870643
4,"TfidfVectorizer(ngram_range=(1, 5))",0.649981,0.639508
3,"CountVectorizer(ngram_range=(1, 5))",0.648604,0.637997
11,"HashingVectorizer(n_features=5000, ngram_range=(1, 5))",0.581934,0.598301
10,"HashingVectorizer(n_features=3000, ngram_range=(1, 5))",0.568540,0.592408


In [13]:
result_model.sort_values('Accuracy', ascending=False)

,Vectorizer,f1_score,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(1, 5))",0.938119,0.939469
1,"TfidfVectorizer(analyzer='char', ngram_range=(1, 5))",0.927087,0.929318
8,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 5))",0.909308,0.912540
7,"HashingVectorizer(analyzer='char', n_features=3000, ngram_range=(1, 5))",0.898584,0.902110
6,"HashingVectorizer(analyzer='char', n_features=2000, ngram_range=(1, 5))",0.892840,0.896243
2,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 5))",0.866822,0.870643
4,"TfidfVectorizer(ngram_range=(1, 5))",0.649981,0.639508
3,"CountVectorizer(ngram_range=(1, 5))",0.648604,0.637997
11,"HashingVectorizer(n_features=5000, ngram_range=(1, 5))",0.581934,0.598301
10,"HashingVectorizer(n_features=3000, ngram_range=(1, 5))",0.568540,0.592408


Символьные векторайзеры все оказались лучше, чем словарные.

### Задание 2.

проверить NER из nltk/spacy/deeppavlov

### NLTK

In [22]:
# nltk.download('tagsets')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('punkt')
# nltk.help.upenn_tagset('RB')
# nltk.help.upenn_tagset('NN')
# nltk.help.upenn_tagset('VB')

In [17]:
records = load_ne5('Collection5/')
document = next(records)
text = document.text
text = re.sub('\r\n\r\n',' ',text)
text = re.sub('\r\n',' ',text)
text

'Обязанности президента Польши будет исполнять Б.Коморовский Обязанности президента Польши вместо погибшего сегодня в авиакатастрофе Леха Качиньского будет исполнять маршал сейма (нижняя палата парламента) Бронислав Коморовский, передают польские СМИ. Сообщается, что именно такой механизм перехода власти предусмотрен Конституцией Польши. В течение двух недель маршал сейма обязан объявить о дате новых президентских выборов на пост главы государства. Ранее сообщалось, что МИД Польши подтвердил гибель президента страны Леха Качиньского в результате авиакатастрофы самолета под Смоленском. По словам пресс-секретаря МИД Польши Петра Пашковского, правительство Польши собирается на экстренное заседание. Сегодня в 10:50 мск в Смоленской области под г.Печерском при заходе на посадку в аэропорт Северный в условиях сильного тумана разбился самолет Ту-154 президента Польши. На борту самолета, по уточненным данным, находились 96 человек, в том числе президент Польши Л.Качиньский и его супруга, все о

In [24]:
document.text = text

In [25]:
nltk.pos_tag(nltk.word_tokenize(text))[:10]

[('Обязанности', 'JJ'),
 ('президента', 'NNP'),
 ('Польши', 'NNP'),
 ('будет', 'NNP'),
 ('исполнять', 'NNP'),
 ('Б.Коморовский', 'NNP'),
 ('Обязанности', 'NNP'),
 ('президента', 'NNP'),
 ('Польши', 'NNP'),
 ('вместо', 'NNP')]

In [27]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') }

{('Бронислав Коморовский', 'PERSON'),
 ('МИД Польши Петра Пашковского', 'ORGANIZATION'),
 ('Польши', 'PERSON'),
 ('Ранее', 'PERSON'),
 ('Сообщается', 'PERSON')}

In [29]:
document.spans[:3]

[Ne5Span(
     index='T1',
     type='GEOPOLIT',
     start=23,
     stop=29,
     text='Польши'
 ),
 Ne5Span(
     index='T2',
     type='PER',
     start=46,
     stop=59,
     text='Б.Коморовский'
 ),
 Ne5Span(
     index='T3',
     type='GEOPOLIT',
     start=86,
     stop=92,
     text='Польши'
 )]

### SpaCy

In [31]:
nlp = spacy.load("ru_core_news_sm")
ny_bb = text
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [35]:
[ (token.text, token.pos_, token.dep_) for token in article[:5] ]

[('Обязанности', 'NOUN', 'nsubj'),
 ('президента', 'NOUN', 'nmod'),
 ('Польши', 'PROPN', 'nmod'),
 ('будет', 'AUX', 'aux'),
 ('исполнять', 'VERB', 'ROOT')]

### написать свой NER, попробовать разные подходы:

- передаём в сетку токен и его соседей.
- передаём в сетку только токен.
- свой вариант.

In [36]:
def get_classification_report(y_test_true, y_test_pred):
    print(classification_report(y_test_true, y_test_pred))

    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [37]:
records = load_ne5('Collection5/')
# next(records)

In [38]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [39]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [40]:
df_words['tag'].value_counts()

tag
OUT         34110
PER          3149
ORG          2224
LOC           777
GEOPOLIT      730
MEDIA         404
Name: count, dtype: int64

In [41]:
df_words.head()

,word,tag
0,Обязанности,OUT
1,президента,OUT
2,Польши,GEOPOLIT
3,будет,OUT
4,исполнять,OUT


In [42]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

Целевая переменная:

In [43]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [44]:
encoder.classes_

array(['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'OUT', 'PER'], dtype=object)

In [45]:
valid_x

864                 г
15055        источник
21626               в
20474       причинами
15816           также
             ...     
1016           работы
32474           этого
19911               "
37856    оппозиционер
39596       заседании
Name: word, Length: 10349, dtype: object

In [46]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [47]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [48]:
def custom_standardization(input_data):
    # Здесь может быть предобработка текста
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(  
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            # ngrams=(1, 3),
                            output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [49]:
len(vectorize_layer.get_vocabulary())

8683

In [50]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax') # [OUT, PER, ORG, LOC, GEOPOLIT, MEDIA]

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [51]:
mmodel = modelNER()

In [52]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [53]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
1941/1941 [==============================] - 29s 14ms/step - loss: 0.4554 - accuracy: 0.8709 - val_loss: 0.3084 - val_accuracy: 0.9153
Epoch 2/3
1941/1941 [==============================] - 28s 15ms/step - loss: 0.1523 - accuracy: 0.9563 - val_loss: 0.2890 - val_accuracy: 0.9299
Epoch 3/3
1941/1941 [==============================] - 28s 14ms/step - loss: 0.0968 - accuracy: 0.9701 - val_loss: 0.2878 - val_accuracy: 0.9309


In [54]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

324/324 [==============================] - 0s 948us/step


In [55]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.9241565722821032

In [56]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.89      0.81      0.84       195
           1       0.92      0.68      0.78       199
           2       0.99      0.65      0.79       107
           3       0.91      0.52      0.66       559
           4       0.93      1.00      0.96      8497
           5       0.98      0.65      0.78       792

    accuracy                           0.93     10349
   macro avg       0.94      0.72      0.80     10349
weighted avg       0.93      0.93      0.92     10349

CONFUSION MATRIX

col_0    0    1   2    3     4    5
row_0                              
0      157    2   0    5    31    0
1        1  136   0    2    60    0
2        0    3  70    2    29    3
3       18    3   0  289   245    4
4        1    4   1   20  8470    1
5        0    0   0    0   280  512


#### Обучим нейронную сеть на биграммах и триграммах

In [57]:
def custom_standardization(input_data):
    # Здесь может быть предобработка текста.
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization( 
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            ngrams=(1, 3),
                            output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [58]:
mmodel = modelNER()

In [59]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [60]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
1941/1941 [==============================] - 29s 15ms/step - loss: 0.4641 - accuracy: 0.8674 - val_loss: 0.3119 - val_accuracy: 0.9140
Epoch 2/3
1941/1941 [==============================] - 29s 15ms/step - loss: 0.1595 - accuracy: 0.9525 - val_loss: 0.2790 - val_accuracy: 0.9298
Epoch 3/3
1941/1941 [==============================] - 28s 14ms/step - loss: 0.0972 - accuracy: 0.9701 - val_loss: 0.2871 - val_accuracy: 0.9299


In [61]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

324/324 [==============================] - 0s 994us/step


In [62]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.9232063420847907

In [63]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.89      0.76      0.82       195
           1       0.89      0.66      0.76       199
           2       0.99      0.77      0.86       107
           3       0.87      0.51      0.65       559
           4       0.93      1.00      0.96      8497
           5       0.98      0.65      0.78       792

    accuracy                           0.93     10349
   macro avg       0.93      0.72      0.80     10349
weighted avg       0.93      0.93      0.92     10349

CONFUSION MATRIX

col_0    0    1   2    3     4    5
row_0                              
0      149    2   0   11    30    3
1        1  131   0    7    60    0
2        0    3  82    2    20    0
3       17    3   0  286   249    4
4        1    8   1   21  8464    2
5        0    0   0    0   280  512
